In [73]:
## InClass2021-10-11

In [74]:
## Q1 Load the data file to your S3 bucket. Using the pandas library, read the csv data file
## and create a data-frame called car price.

import pandas as pd
import boto3
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
## for regression is RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score


## Defining the S3 bucket
s3 = boto3.resource('s3')
bucket_name = 'bonnieh-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the file to be read from s3 bucket
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

##R Reading the csv file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [75]:
## Q2 Remove observations with missing values.

heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [76]:
## Q3  Using age, currentSmoker, totChol, BMI, and heartRate as the predictor variables, and
## TenYearCHD is the target variable, split the data in

## defining input and target variables
X = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
Y = heart['TenYearCHD']

## evaluate if data is unbalanced--it is, so need to stratify
Y.value_counts()/Y.shape[0]

## splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)



In [77]:
Y_train.value_counts()/Y_train.shape[0]

0    0.847811
1    0.152189
Name: TenYearCHD, dtype: float64

In [78]:
Y_test.value_counts()/Y_test.shape[0]

0    0.846995
1    0.153005
Name: TenYearCHD, dtype: float64

In [79]:
## Q4 Transform each of the input variables in the train and test dataset to 0-1 scale.

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [80]:
## Q5 Using the train data-frame, build a logistic regression model in which: age, currentSmoker,
## totChol, BMI, and heartRate are the predictor variables, and TenYearCHD is the target variable. Using
## this model, predict the likelihood of risk of coronary disease of the patients in the test data-frame. 
## Using 10% as cutoff value, report the recall.

md1 = LogisticRegression().fit(X_train, Y_train)

## predicting on test
pred1 = md1.predict_proba(X_test)[:,1]
pred1 = np.where(pred1 < 0.1, 0, 1)

## computing the recall
recall_score(Y_test, pred1)

0.8214285714285714

In [81]:
## Q6 Using the train data-frame, build a random forest model (with 500 trees and max tree depth
## equal 5) in which: age, currentSmoker, totChol, BMI, and heartRate are the predictor variables,
## and TenYearCHD is the target variable. Using this model, predict the risk of coronary disease of the
##patients in the test data-frame. Using 10% as cutoff value, report the recall.

## building the Random Forest model
md2 = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train, Y_train)

## making preductions using the RF model
pred2 = md2.predict_proba(X_test)[:,1]
pred2 = np.where(pred2 < 0.1, 0, 1)

## Calculate the recall score
recall_score(Y_test, pred2)

0.8392857142857143

In [ ]:
## Q7 Using the results from parts (5) and (6), what model would you use to predict TenYearCHD
## based on the recall?

## Based on the most recent run, I would pick the random forest model because it has a slightly higher
## recall value. But repeat runs makes it clear that the split changes the outcome 
## since the models are so close in their recall values.